### DB Analysis

In [2]:
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from database_handler import DatabaseHandler

In [3]:
db = DatabaseHandler('../data/database/stock_db.db')

In [4]:
db.fetch_stock().sort(by=['last_update'])

tic,name,sector,last_update,spx_status,active
str,str,str,date,i64,i64
"""ABI""",null,"""Industrials""",2005-01-01,0,0
"""ABKFQ""",null,"""Financials""",2005-01-01,0,0
"""ABS""",null,"""Consumer Staples""",2005-01-01,0,0
"""ACV""",null,"""Consumer Staples""",2005-01-01,0,0
"""ANRZQ""",null,"""Materials""",2005-01-01,0,0
…,…,…,…,…,…
"""ZBH""","""Zimmer Biomet""","""Health Care""",2024-10-24,1,1
"""ZTS""","""Zoetis""","""Health Care""",2024-10-24,1,1
"""ERIE""","""Erie Indemnity""","""Financials""",2024-10-24,1,1


In [29]:
info = db.fetch_info().to_pandas()
info

34.97732519317698

In [11]:
mdf = db.fetch_market_data("SOLV")
mdf

tic,date,close,adj_close,volume
str,date,f64,f64,i64
"""SOLV""",2024-04-01,69.099998,69.099998,14094900
"""SOLV""",2024-04-02,66.900002,66.900002,9446900
"""SOLV""",2024-04-03,63.889999,63.889999,5177200
"""SOLV""",2024-04-04,67.0,67.0,5239700
"""SOLV""",2024-04-05,70.059998,70.059998,4216300
…,…,…,…,…
"""SOLV""",2024-10-17,72.410004,72.410004,1252200
"""SOLV""",2024-10-18,73.68,73.68,831700
"""SOLV""",2024-10-21,73.099998,73.099998,834900


In [10]:
fdf = db.fetch_financial_data("SOLV")
fdf

tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct
str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64
"""SOLV""",2023-03-31,2023-06-19,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SOLV""",2023-06-30,2023-09-18,2076.0,887.0,579.0,321.0,546.0,172.709,2718.0,14311.0,500.0,1194.0,863.0,1448.0,1412.0,8317.0,10347.0,null,3964.0,366.0,-71.0,-299.0,null,71.0,12281.0,null
"""SOLV""",2023-09-30,2023-12-19,2054.0,901.0,581.0,285.0,537.0,172.709,2656.0,13873.0,500.0,1153.0,849.0,1371.0,1471.0,8325.0,10623.0,null,3250.0,493.0,-6.0,-485.0,null,66.0,11575.0,null
"""SOLV""",2023-12-31,2024-03-20,2095.0,980.0,575.0,73.0,489.0,172.711,2916.0,14377.0,600.0,1293.0,869.0,1466.0,1771.0,8303.0,10968.0,null,3409.0,547.0,-88.0,-306.0,null,88.0,11712.0,null
"""SOLV""",2024-03-31,2024-05-09,2016.0,844.0,596.0,237.0,507.0,172.71,3359.0,14711.0,996.0,1220.0,901.0,1491.0,1854.0,8303.0,10860.0,null,3851.0,442.0,-102.0,462.0,null,102.0,12154.0,0.192
"""SOLV""",2024-06-30,2024-08-08,2081.0,944.0,701.0,89.0,343.0,172.711,3363.0,14578.0,897.0,1317.0,899.0,1537.0,2571.0,8306.0,11711.0,89.0,2867.0,355.0,-58.0,-396.0,null,58.0,11173.0,0.0365


In [12]:
db.fetch_insider_data("AAPL").to_pandas()

,tic,filling_date,trade_date,owner_name,title,transaction_type,last_price,qty,shares_held,owned,value
0,AAPL,2004-04-21,2004-04-19,Anderson Fred D,EVP,S - Sale+OE,$27.90,"-44,200","2,672",-94%,"-$1,233,165"
1,AAPL,2004-04-21,2004-04-19,Anderson Fred D,EVP,S - Sale+OE,$28.00,"-54,500","2,672",-95%,"-$1,526,068"
2,AAPL,2004-04-21,2004-04-20,Anderson Fred D,EVP,S - Sale+OE,$28.27,"-64,000","2,672",-96%,"-$1,809,460"
3,AAPL,2004-04-21,2004-04-19,Anderson Fred D,EVP,S - Sale+OE,$28.24,"-70,000","2,672",-96%,"-$1,976,480"
4,AAPL,2004-04-21,2004-04-19,Anderson Fred D,EVP,S - Sale+OE,$28.38,"-71,300","2,672",-96%,"-$2,023,715"
...,...,...,...,...,...,...,...,...,...,...,...
595,AAPL,2024-10-03,2024-10-02,Adams Katherine L.,"SVP, GC, Secretary",S - Sale+OE,$226.20,"-61,019","187,043",-25%,"-$13,802,297"
596,AAPL,2024-10-03,2024-10-02,Cook Timothy D,CEO,S - Sale+OE,$224.46,"-223,986","3,280,180",-6%,"-$50,276,355"
597,AAPL,2024-10-03,2024-10-02,O'Brien Deirdre,SVP,S - Sale+OE,$226.87,"-61,019","136,572",-31%,"-$13,843,382"
598,AAPL,2024-10-03,2024-10-02,Williams Jeffrey E,COO,S - Sale+OE,$226.86,"-59,730","489,944",-11%,"-$13,550,148"


In [7]:
mdf = db.fetch_market_data("PLTR")
mdf.to_pandas()

,tic,date,close,adj_close,volume
0,PLTR,2020-09-30,9.500000,9.500000,338584400
1,PLTR,2020-10-01,9.460000,9.460000,124297600
2,PLTR,2020-10-02,9.200000,9.200000,55018300
3,PLTR,2020-10-05,9.030000,9.030000,36316900
4,PLTR,2020-10-06,9.900000,9.900000,90864000
...,...,...,...,...,...
1010,PLTR,2024-10-07,38.889999,38.889999,44124800
1011,PLTR,2024-10-08,41.450001,41.450001,62542900
1012,PLTR,2024-10-09,43.130001,43.130001,86404700
1013,PLTR,2024-10-10,43.520000,43.520000,62516300


In [8]:
st = db.fetch_stock().to_pandas()
st

,tic,name,sector,last_update,spx_status,active
0,AABA,None,Financials,2017-04-18,0,0
1,AAP,None,Consumer Discretionary,2024-02-28,0,1
2,ABC,None,Health Care,2023-08-02,0,0
3,ABI,None,Industrials,2005-01-01,0,0
4,ABKFQ,None,Financials,2005-01-01,0,0
...,...,...,...,...,...,...
935,ZTS,Zoetis,Health Care,2024-10-13,1,1
936,AMTM,Amentum,Industrials,2005-01-01,1,0
937,ERIE,Erie Indemnity,Financials,2024-10-13,1,1
938,PLTR,Palantir Technologies,Information Technology,2024-10-13,1,1


In [9]:
inf = db.fetch_info("AAPL").to_pandas()
inf

,tic,risk,beta,volume,trailing_pe,forward_pe,trailing_peg,peg,trailing_eps,forward_eps,...,currency,shares_outstanding,enterprise_value,short_ratio,curr_price,fiftytwo_wc,rec_key,target_low,target_high,target_mean
0,AAPL,1,1.239,30817675.0,34.58207,30.421124,2.1839,3.09,6.58,7.48,...,USD,1.520410e+10,3.499204e+12,2.32,227.55,0.273221,None,183.86,300.0,240.78


In [10]:
inf.loc[0, ["risk"]].values[0]

1